# Sales Forecasting with RasgoQL

This notebook shows how to perform the data preparation and feature engineering for a sales forecasting model. Starting with [AdventureWorks](https://docs.microsoft.com/en-us/sql/samples/adventureworks-install-configure) data loaded in a Snowflake data warehouse, the data will be explored, features created and modeling data extracted.

This analysis will be focused on the internet sales for this company.

## Packages

The documentation for each packaged used in this tutorial is linked below:
* [configparser](https://docs.python.org/3/library/configparser.html)
* [numpy](https://numpy.org/doc/stable/)
* [os](https://docs.python.org/3/library/os.html)
* [pandas](https://pandas.pydata.org/docs/)
* [rasgoQL](https://docs.rasgoql.com/)
* [scikit-learn](https://scikit-learn.org/stable/)
    * [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)
* [XGBoost](https://xgboost.readthedocs.io/en/latest/)

In [1]:
import configparser
import numpy as np
import os
import pandas as pd
import rasgoql
from sklearn.metrics import mean_squared_error
import xgboost as xgb

## Upload AdventureWorks data to Snowflake

See [here](https://medium.com/@griffatrasgo/1946cb9930ea) to add AdventureWorks data to Snowflake.

## Work with RasgoQL

### Get your Snowflake credentials and connect to Snowflake

In [2]:
snowcred = configparser.ConfigParser()
snowcred.read("snowcred.cfg")
account = snowcred['AW']['account']
user = snowcred['AW']['user']
password = snowcred['AW']['password']
role = snowcred['AW']['role']
warehouse = snowcred['AW']['warehouse']
database = snowcred['AW']['database']
schema = snowcred['AW']['schema']

### Connect to Snowflake

In [3]:
creds = rasgoql.SnowflakeCredentials(account=account,
                                     user=user,
                                     password=password,
                                     role=role,
                                     warehouse=warehouse,
                                     database=database,
                                     schema=schema)

In [4]:
rql = rasgoql.connect(credentials=creds)

### Get list of available datasets

List all available tables in the ADVENTUREWORKS database.

In [5]:
rql.list_tables('ADVENTUREWORKS')

,TABLE_NAME,FQTN,TABLE_TYPE,ROW_COUNT,CREATED,LAST_ALTERED
0,ADVENTUREWORKSDWBUILDVERSION,ADVENTUREWORKS.PUBLIC.ADVENTUREWORKSDWBUILDVER...,TABLE,0,2021-11-06 09:07:48.039000-07:00,2022-02-09 00:00:20.470000-08:00
1,DIMACCOUNT,ADVENTUREWORKS.PUBLIC.DIMACCOUNT,TABLE,99,2021-11-06 09:07:48.012000-07:00,2022-02-09 00:00:20.470000-08:00
2,DIMCURRENCY,ADVENTUREWORKS.PUBLIC.DIMCURRENCY,TABLE,105,2021-11-06 09:07:48.071000-07:00,2022-02-09 00:00:20.470000-08:00
3,DIMCUSTOMER,ADVENTUREWORKS.PUBLIC.DIMCUSTOMER,TABLE,18484,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.470000-08:00
4,DIMDATE,ADVENTUREWORKS.PUBLIC.DIMDATE,TABLE,3652,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.470000-08:00
5,DIMDEPARTMENTGROUP,ADVENTUREWORKS.PUBLIC.DIMDEPARTMENTGROUP,TABLE,7,2021-11-06 09:07:48.071000-07:00,2022-02-09 00:00:20.470000-08:00
6,DIMEMPLOYEE,ADVENTUREWORKS.PUBLIC.DIMEMPLOYEE,TABLE,296,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.471000-08:00
7,DIMGEOGRAPHY,ADVENTUREWORKS.PUBLIC.DIMGEOGRAPHY,TABLE,655,2021-11-06 09:07:48.037000-07:00,2022-02-09 00:00:20.471000-08:00
8,DIMORGANIZATION,ADVENTUREWORKS.PUBLIC.DIMORGANIZATION,TABLE,14,2021-11-06 09:07:48.115000-07:00,2022-02-09 00:00:20.470000-08:00
9,DIMPRODUCT,ADVENTUREWORKS.PUBLIC.DIMPRODUCT,TABLE,606,2021-11-06 09:07:47.965000-07:00,2022-02-09 00:00:20.470000-08:00


The three tables of interest are ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES containing the sales information, ADVENTUREWORKS.PUBLIC.DIMPROMOTION containing details about any promotions in place and ADVENTUREWORKS.PUBLIC.DIMPRODUCT containing details about the products.

Examine the transforms that RasgoQL can apply to these tables

In [6]:
rql.list_transforms()

[RasgoTemplate: one_hot_encode(column: column),
 RasgoTemplate: datetrunc(dates: datepart_dict),
 RasgoTemplate: datepart(dates: datepart_dict),
 RasgoTemplate: concat(concat_list: mixed_list, name: value),
 RasgoTemplate: train_test_split(order_by: column_list, train_percent: value),
 RasgoTemplate: standard_scaler(columns_to_scale: column_list, averages: value_list, standarddevs: value_list),
 RasgoTemplate: bin(type: string, bin_count: int, column: column),
 RasgoTemplate: to_date(dates: column_value_dict),
 RasgoTemplate: cast(casts: cast_value_dict),
 RasgoTemplate: label_encode(column: column),
 RasgoTemplate: impute(imputations: imputation_dict, flag_missing_vals: boolean),
 RasgoTemplate: math(math_ops: math_list, names: value_list),
 RasgoTemplate: min_max_scaler(columns_to_scale: column_list, minimums: value_list, maximums: value_list),
 RasgoTemplate: substring(target_col: column, start_pos: value, end_pos: value),
 RasgoTemplate: target_encode(column: column, target: column

### Examine Internet Sales

In [7]:
internet_sales = rql.dataset('ADVENTUREWORKS.PUBLIC.FACTINTERNETSALES')
internet_sales.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,1898.0944,1898.0944,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07


This looks promising, but I'd like to see a single product sorted by date. This can be done through the use of the filter and order transforms. To use filter, the product we want to filter on is needed, as we don't know that yet, we will just order by *PRODUCTKEY* and *ORDERDATE*.

In [8]:
internet_sales.order(order_by={'PRODUCTKEY':'ASC','ORDERDATE':'ASC'}).preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,214,20121228,20130109,20130104,16313,1,100,8,SO51180,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,214,20121228,20130109,20130104,12132,1,100,7,SO51181,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,214,20121229,20130110,20130105,12390,1,100,8,SO51191,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,214,20121229,20130110,20130105,11241,1,100,7,SO51192,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,214,20121230,20130111,20130106,11338,1,100,8,SO51207,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,214,20121230,20130111,20130106,24604,1,6,9,SO51212,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,214,20121231,20130112,20130107,28204,1,6,9,SO51234,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,214,20121231,20130112,20130107,11615,1,98,10,SO51232,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,214,20121231,20130112,20130107,11061,1,6,9,SO51237,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,214,20121231,20130112,20130107,25625,1,100,8,SO51246,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


This looks reasonable, use this for our modeling. For future reference, what columns exist in this table?

In [9]:
internet_sales.preview().columns.sort_values()

Index(['CARRIERTRACKINGNUMBER', 'CURRENCYKEY', 'CUSTOMERKEY',
       'CUSTOMERPONUMBER', 'DISCOUNTAMOUNT', 'DUEDATE', 'DUEDATEKEY',
       'EXTENDEDAMOUNT', 'FREIGHT', 'ORDERDATE', 'ORDERDATEKEY',
       'ORDERQUANTITY', 'PRODUCTKEY', 'PRODUCTSTANDARDCOST', 'PROMOTIONKEY',
       'REVISIONNUMBER', 'SALESAMOUNT', 'SALESORDERLINENUMBER',
       'SALESORDERNUMBER', 'SALESTERRITORYKEY', 'SHIPDATE', 'SHIPDATEKEY',
       'TAXAMT', 'TOTALPRODUCTCOST', 'UNITPRICE', 'UNITPRICEDISCOUNTPCT'],
      dtype='object')

Interesting fields that may link back to other tables: *CURRENCYKEY*, *CUSTOMERKEY*, *PRODUCTKEY*, *PROMOTIONKEY*, *SALESTERRITORYKEY*.

Not all of these are relevant, but *PRODUCTKEY*, *PROMOTIONKEY* are probably important for a sales forecast. To find which datasets we can find these in, pull the list of datasets and look for adventureworks

### Examine Product and Promotion Data

Table ADVENTUREWORKS.PUBLIC.DIMPROMOTION looks like it will contain information on the promotion and ADVENTUREWORKS.PUBLIC.DIMPRODUCT on the product. Take a look at the first.

In [10]:
promotion = rql.dataset('ADVENTUREWORKS.PUBLIC.DIMPROMOTION')
promotion.preview()

,PROMOTIONKEY,PROMOTIONALTERNATEKEY,ENGLISHPROMOTIONNAME,SPANISHPROMOTIONNAME,FRENCHPROMOTIONNAME,DISCOUNTPCT,ENGLISHPROMOTIONTYPE,SPANISHPROMOTIONTYPE,FRENCHPROMOTIONTYPE,ENGLISHPROMOTIONCATEGORY,SPANISHPROMOTIONCATEGORY,FRENCHPROMOTIONCATEGORY,STARTDATE,ENDDATE,MINQTY,MAXQTY
0,1,1,No Discount,Sin descuento,Aucune remise,0.00,No Discount,Sin descuento,Aucune remise,No Discount,Sin descuento,Aucune remise,2010-11-29,2014-06-30,0,NaN
1,2,2,Volume Discount 11 to 14,Descuento por volumen (entre 11 y 14),Remise sur quantité (de 11 à 14),0.02,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,11,14.0
2,3,3,Volume Discount 15 to 24,Descuento por volumen (entre 15 y 24),Remise sur quantité (de 15 à 24),0.05,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,15,24.0
3,4,4,Volume Discount 25 to 40,Descuento por volumen (entre 25 y 40),Remise sur quantité (de 25 à 40),0.10,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,25,40.0
4,5,5,Volume Discount 41 to 60,Descuento por volumen (entre 41 y 60),Remise sur quantité (de 41 à 60),0.15,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,41,60.0
5,6,6,Volume Discount over 60,Descuento por volumen (más de 60),Remise sur quantité (au-delà de 60),0.20,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,61,NaN
6,7,7,Mountain-100 Clearance Sale,"Liquidación de bicicleta de montaña, 100",Liquidation VTT 100,0.35,Discontinued Product,Descatalogado,Ce produit n'est plus commercialisé,Reseller,Distribuidor,Revendeur,2011-11-12,2011-12-28,0,NaN
7,8,8,Sport Helmet Discount-2002,"Casco deportivo, descuento: 2002",Remise sur les casques sport - 2002,0.10,Seasonal Discount,Descuento de temporada,Remise saisonnière,Reseller,Distribuidor,Revendeur,2011-12-29,2012-01-28,0,NaN
8,9,9,Road-650 Overstock,"Bicicleta de carretera: 650, oferta especial",Déstockage Vélo de route 650,0.30,Excess Inventory,Inventario excedente,Déstockage,Reseller,Distribuidor,Revendeur,2011-12-29,2012-02-28,0,NaN
9,10,10,Mountain Tire Sale,Oferta de cubierta de montaña,Vente de pneus de VTT,0.50,Excess Inventory,Inventario excedente,Déstockage,Customer,Cliente,Client,2012-12-12,2013-02-26,0,NaN


In [11]:
promotion.preview().columns.sort_values()

Index(['DISCOUNTPCT', 'ENDDATE', 'ENGLISHPROMOTIONCATEGORY',
       'ENGLISHPROMOTIONNAME', 'ENGLISHPROMOTIONTYPE',
       'FRENCHPROMOTIONCATEGORY', 'FRENCHPROMOTIONNAME', 'FRENCHPROMOTIONTYPE',
       'MAXQTY', 'MINQTY', 'PROMOTIONALTERNATEKEY', 'PROMOTIONKEY',
       'SPANISHPROMOTIONCATEGORY', 'SPANISHPROMOTIONNAME',
       'SPANISHPROMOTIONTYPE', 'STARTDATE'],
      dtype='object')

And the product dataset.

In [12]:
product = rql.dataset('ADVENTUREWORKS.PUBLIC.DIMPRODUCT')
product.preview()

,PRODUCTKEY,PRODUCTALTERNATEKEY,PRODUCTSUBCATEGORYKEY,WEIGHTUNITMEASURECODE,SIZEUNITMEASURECODE,ENGLISHPRODUCTNAME,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,STANDARDCOST,FINISHEDGOODSFLAG,...,CHINESEDESCRIPTION,ARABICDESCRIPTION,HEBREWDESCRIPTION,THAIDESCRIPTION,GERMANDESCRIPTION,JAPANESEDESCRIPTION,TURKISHDESCRIPTION,STARTDATE,ENDDATE,STATUS
0,1,AR-5381,NaN,None,None,Adjustable Race,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
1,2,BA-8327,NaN,None,None,Bearing Ball,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
2,3,BE-2349,NaN,None,None,BB Ball Bearing,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
4,5,BL-2036,NaN,None,None,Blade,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
5,6,CA-5965,NaN,None,None,LL Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
6,7,CA-6738,NaN,None,None,ML Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
7,8,CA-7457,NaN,None,None,HL Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
8,9,CB-2903,NaN,None,None,Chainring Bolts,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
9,10,CN-6137,NaN,None,None,Chainring Nut,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current


In [13]:
product.preview().columns.sort_values()

Index(['ARABICDESCRIPTION', 'CHINESEDESCRIPTION', 'CLASS', 'COLOR',
       'DAYSTOMANUFACTURE', 'DEALERPRICE', 'ENDDATE', 'ENGLISHDESCRIPTION',
       'ENGLISHPRODUCTNAME', 'FINISHEDGOODSFLAG', 'FRENCHDESCRIPTION',
       'FRENCHPRODUCTNAME', 'GERMANDESCRIPTION', 'HEBREWDESCRIPTION',
       'JAPANESEDESCRIPTION', 'LARGEPHOTO', 'LISTPRICE', 'MODELNAME',
       'PRODUCTALTERNATEKEY', 'PRODUCTKEY', 'PRODUCTLINE',
       'PRODUCTSUBCATEGORYKEY', 'REORDERPOINT', 'SAFETYSTOCKLEVEL', 'SIZE',
       'SIZERANGE', 'SIZEUNITMEASURECODE', 'SPANISHPRODUCTNAME',
       'STANDARDCOST', 'STARTDATE', 'STATUS', 'STYLE', 'THAIDESCRIPTION',
       'TURKISHDESCRIPTION', 'WEIGHT', 'WEIGHTUNITMEASURECODE'],
      dtype='object')

## Sales Data

Work with the sales and promotion data to create the base modeling time-series features for the sales forecasting model.

### Merge Promo data

First, we want to clean up the promotion data to only keep what needs to be added to the sales data. Drop all columns except *PROMOTIONKEY* and *DISCOUNTPCT* from  promotion using the `drop_columns` transformation.

In [14]:
reduced_promo = promotion.drop_columns(include_cols=['PROMOTIONKEY', 'DISCOUNTPCT'])
reduced_promo.order(order_by={'PROMOTIONKEY':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT
0,1,0.00
1,2,0.02
2,3,0.05
3,4,0.10
4,5,0.15
5,6,0.20
6,7,0.35
7,8,0.10
8,9,0.30
9,10,0.50


Now merge this with the internet sales datausing the `join` transformation.

In [15]:
sales_promo = reduced_promo.join(join_table=internet_sales.fqtn,
                                 join_type='RIGHT',
                                 join_columns={'PROMOTIONKEY':'PROMOTIONKEY'})
sales_promo.order(order_by={'PRODUCTKEY':'ASC','ORDERDATE':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,1,0.0,214,20121228,20130109,20130104,16313,100,8,SO51180,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,1,0.0,214,20121228,20130109,20130104,12132,100,7,SO51181,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,1,0.0,214,20121229,20130110,20130105,12390,100,8,SO51191,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,1,0.0,214,20121229,20130110,20130105,11241,100,7,SO51192,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,1,0.0,214,20121230,20130111,20130106,11338,100,8,SO51207,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,1,0.0,214,20121230,20130111,20130106,24604,6,9,SO51212,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,1,0.0,214,20121231,20130112,20130107,28204,6,9,SO51234,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,1,0.0,214,20121231,20130112,20130107,11615,98,10,SO51232,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,1,0.0,214,20121231,20130112,20130107,11061,6,9,SO51237,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,1,0.0,214,20121231,20130112,20130107,25625,100,8,SO51246,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


### Create Weekly Data

Now, we want to forecast these sales weekly, so we need to extract the week from the *ORDERDATE*. This can be done using the transform `datetrunc`.

In [16]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'})
salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERDATE':'ASC'}).preview()

,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,...,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE,ORDERDATE_WEEK
0,1,0.0,214,20121228,20130109,20130104,16313,100,8,SO51180,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04,2012-12-23
1,1,0.0,214,20121228,20130109,20130104,12132,100,7,SO51181,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04,2012-12-23
2,1,0.0,214,20121229,20130110,20130105,12390,100,8,SO51191,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05,2012-12-23
3,1,0.0,214,20121229,20130110,20130105,11241,100,7,SO51192,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05,2012-12-23
4,1,0.0,214,20121230,20130111,20130106,11338,100,8,SO51207,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06,2012-12-30
5,1,0.0,214,20121230,20130111,20130106,24604,6,9,SO51212,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06,2012-12-30
6,1,0.0,214,20121231,20130112,20130107,28204,6,9,SO51234,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30
7,1,0.0,214,20121231,20130112,20130107,11615,98,10,SO51232,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30
8,1,0.0,214,20121231,20130112,20130107,11061,6,9,SO51237,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30
9,1,0.0,214,20121231,20130112,20130107,25625,100,8,SO51246,...,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07,2012-12-30


The new week column is called *ORDERDATE_WEEK*. This is clunky, so let's rename it to *ORDERWEEK* using the `rename` transformation.

In [17]:
newsalesds = salesds.rename(renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
newsalesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ORDERWEEK,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,2012-12-23,1,0.0,214,20121228,20130109,20130104,16313,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,2012-12-23,1,0.0,214,20121228,20130109,20130104,12132,100,7,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,2012-12-23,1,0.0,214,20121229,20130110,20130105,11241,100,7,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,2012-12-23,1,0.0,214,20121229,20130110,20130105,12390,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,2012-12-30,1,0.0,214,20121230,20130111,20130106,24604,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,2012-12-30,1,0.0,214,20121230,20130111,20130106,11338,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,2012-12-30,1,0.0,214,20121231,20130112,20130107,11061,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,2012-12-30,1,0.0,214,20121231,20130112,20130107,25625,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,2012-12-30,1,0.0,214,20121231,20130112,20130107,11615,98,10,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,2012-12-30,1,0.0,214,20121231,20130112,20130107,28204,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


Alternatively, we can just chain these transformations together.

In [18]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ORDERWEEK,PROMOTIONKEY,DISCOUNTPCT,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,CURRENCYKEY,SALESTERRITORYKEY,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,2012-12-23,1,0.0,214,20121228,20130109,20130104,16313,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,2012-12-23,1,0.0,214,20121228,20130109,20130104,12132,100,7,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,2012-12-23,1,0.0,214,20121229,20130110,20130105,11241,100,7,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,2012-12-23,1,0.0,214,20121229,20130110,20130105,12390,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,2012-12-30,1,0.0,214,20121230,20130111,20130106,24604,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,2012-12-30,1,0.0,214,20121230,20130111,20130106,11338,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,2012-12-30,1,0.0,214,20121231,20130112,20130107,11061,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,2012-12-30,1,0.0,214,20121231,20130112,20130107,25625,100,8,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,2012-12-30,1,0.0,214,20121231,20130112,20130107,11615,98,10,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,2012-12-30,1,0.0,214,20121231,20130112,20130107,28204,6,9,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


Now we can aggregate this to the product-week level and create aggregations of the *'DISCOUNTAMOUNT'*, *'DISCOUNTPCT'*, *'ORDERQUANTITY'*, *'PRODUCTSTANDARDCOST'*, *'SALESAMOUNT'*, *'TAXAMT'*, *'TOTALPRODUCTCOST'*, *'UNITPRICE'*, *'UNITPRICEDISCOUNTPCT'* using the `aggregate` transform.

In [19]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': ['MIN', 'MAX', 'AVG', 'SUM'], 
                                              'DISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']})
salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,SALESAMOUNT_SUM,TAXAMT_SUM,TOTALPRODUCTCOST_AVG,TOTALPRODUCTCOST_SUM,UNITPRICE_AVG,UNITPRICE_SUM,UNITPRICEDISCOUNTPCT_MIN,UNITPRICEDISCOUNTPCT_MAX,UNITPRICEDISCOUNTPCT_AVG,UNITPRICEDISCOUNTPCT_SUM
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,139.9600,11.1968,13.0863000000,52.3452,34.9900000000,139.9600,0.0,0.0,0.0,0.0
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,349.9000,27.9920,13.0863000000,130.8630,34.9900000000,349.9000,0.0,0.0,0.0,0.0
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.8800,33.5904,13.0863000000,157.0356,34.9900000000,419.8800,0.0,0.0,0.0,0.0
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,209.9400,16.7952,13.0863000000,78.5178,34.9900000000,209.9400,0.0,0.0,0.0,0.0
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,384.8900,30.7912,13.0863000000,143.9493,34.9900000000,384.8900,0.0,0.0,0.0,0.0
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1084.6900,86.7752,13.0863000000,405.6753,34.9900000000,1084.6900,0.0,0.0,0.0,0.0
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1539.5600,123.1648,13.0863000000,575.7972,34.9900000000,1539.5600,0.0,0.0,0.0,0.0
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1189.6600,95.1728,13.0863000000,444.9342,34.9900000000,1189.6600,0.0,0.0,0.0,0.0
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1154.6700,92.3736,13.0863000000,431.8479,34.9900000000,1154.6700,0.0,0.0,0.0,0.0
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1294.6300,103.5704,13.0863000000,484.1931,34.9900000000,1294.6300,0.0,0.0,0.0,0.0


This gives us statistics for each product over a given week.

### Time-series feature engineering

For sales forcasting, in addition to the lagged variables, we need to know what the sales were in prior weeks. The transform `lag` can create these variables for us. In this case we will lag the following variables *'DISCOUNTAMOUNT_AVG'*, *'DISCOUNTPCT_AVG'*, *'ORDERQUANTITY_SUM'*, *'PRODUCTSTANDARDCOST_AVG'*, *'SALESAMOUNT_SUM'*, *'TAXAMT_SUM'*, *'TOTALPRODUCTCOST_SUM'*,*'UNITPRICEDISCOUNTPCT_AVG'*, *'UNITPRICE_AVG'*, *'UNITPRICE_SUM'*
over *1*, *2*, *3*, and *12* weeks.

In [20]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': ['MIN', 'MAX', 'AVG', 'SUM'], 
                                              'DISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']}).lag(
                                columns=['DISCOUNTAMOUNT_AVG', 'DISCOUNTPCT_AVG', 'ORDERQUANTITY_SUM', 
                                         'PRODUCTSTANDARDCOST_AVG', 'SALESAMOUNT_SUM', 'TAXAMT_SUM', 
                                         'TOTALPRODUCTCOST_SUM','UNITPRICEDISCOUNTPCT_AVG', 
                                         'UNITPRICE_AVG', 'UNITPRICE_SUM'],
                                amounts=[1, 2, 3, 12],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY'])
   
salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICEDISCOUNTPCT_AVG_3,LAG_UNITPRICEDISCOUNTPCT_AVG_12,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,None,None,None,None,None,None,None,None
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34.9900000000,None,None,None,139.9600,None,None,None
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None


In addition to lag variables, the moving average of the quantites can be useful. In this case, we'll calculate the moving average over *4* observations of *ORDERQUANTITY_SUM* and *TOTALPRODUCTCOST_SUM* using the transform `moving_avg`.

In [21]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': ['MIN', 'MAX', 'AVG', 'SUM'], 
                                              'DISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']}).lag(
                                columns=['DISCOUNTAMOUNT_AVG', 'DISCOUNTPCT_AVG', 'ORDERQUANTITY_SUM', 
                                         'PRODUCTSTANDARDCOST_AVG', 'SALESAMOUNT_SUM', 'TAXAMT_SUM', 
                                         'TOTALPRODUCTCOST_SUM','UNITPRICEDISCOUNTPCT_AVG', 
                                         'UNITPRICE_AVG', 'UNITPRICE_SUM'],
                                amounts=[1, 2, 3, 12],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY']).moving_avg(
                                input_columns=['ORDERQUANTITY_SUM', 'SALESAMOUNT_SUM'],
                                window_sizes=[4],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY'])
    
salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,4.000,139.9600000
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,None,None,None,139.9600,None,None,None,7.000,244.9300000
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None,8.666,303.2466666
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None,8.000,279.9200000
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None,9.750,341.1525000
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None,15.000,524.8500000
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None,23.000,804.7700000
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None,30.000,1049.7000000
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None,35.500,1242.1450000
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None,37.000,1294.6300000


#### Save result

At this point, the data has been aggregated to weekly data and multiple transformations have been applied. This could be a good starting point for additional analysis and useful for visualization. For this reason, we will publish it to Rasgo to make it available for others to use. This can be done with the `rasgo.publish.dataset` function

In [22]:
weeklysales = salesds.save(table_name="WKSP_AW_WEEKLY_SALES",
                           table_type="VIEW")
weeklysales

Dataset(fqtn=RASGOQL.PUBLIC.WKSP_AW_WEEKLY_SALES, type=view)

#### Capture trends

Lag variables are necessary for time-series models, but often calculating trend variables provides additional value. These can be simple differences or ratios, more complicated ratios such as the difference between two lags divided by the time between the observations (velocity) or a weighted mocing average (often providing more weight to the most recent observations. All of these can be calculated using the `math` transformation. In this case, we will calculate only

* *ORDERQUANTITY_SUM - LAG_ORDERQUANTITY_SUM_3*
* *ORDERQUANTITY_SUM / LAG_ORDERQUANTITY_SUM_3*
* *(SALESAMOUNT_SUM - LAG_SALESAMOUNT_SUM_3) / 4*
* *SALESAMOUNT_SUM / MEAN_SALESAMOUNT_SUM_4*
* *(4*SALESAMOUNT_SUM + 3*LAG_SALESAMOUNT_SUM_1 + 2*LAG_SALESAMOUNT_SUM_1 + LAG_SALESAMOUNT_SUM_3)/10*

In [23]:
salesds = weeklysales.math(math_ops=['ORDERQUANTITY_SUM - LAG_ORDERQUANTITY_SUM_3',
                                     'ORDERQUANTITY_SUM / NULLIF(LAG_ORDERQUANTITY_SUM_12, 0)',
                                     '(SALESAMOUNT_SUM - LAG_SALESAMOUNT_SUM_3) / 4',
                                     'SALESAMOUNT_SUM / NULLIF(MEAN_SALESAMOUNT_SUM_4, 0)',
                                     '(4*SALESAMOUNT_SUM + 3*LAG_SALESAMOUNT_SUM_1 + 2*LAG_SALESAMOUNT_SUM_1 + LAG_SALESAMOUNT_SUM_3)/10'])
salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,ORDERQUANTITY_SUM___LAG_ORDERQUANTITY_SUM_3,ORDERQUANTITY_SUM__NULLIFLAG_ORDERQUANTITY_SUM_12_0,SALESAMOUNT_SUM___LAG_SALESAMOUNT_SUM_3__4,SALESAMOUNT_SUM__NULLIFMEAN_SALESAMOUNT_SUM_4_0,_4SALESAMOUNT_SUM__3LAG_SALESAMOUNT_SUM_1__2LAG_SALESAMOUNT_SUM_1__LAG_SALESAMOUNT_SUM_310
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,4.000,139.9600000,NaN,NaN,None,1.0000000000,None
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,7.000,244.9300000,NaN,NaN,None,1.4285714286,None
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,139.9600,None,None,8.666,303.2466666,NaN,NaN,None,1.3846153849,None
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,349.9000,139.9600,None,8.000,279.9200000,2.0,NaN,17.4950000000,0.7500000000,307.9120000000
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.8800,349.9000,None,9.750,341.1525000,1.0,NaN,8.7475000000,1.1282051282,293.9160000000
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,209.9400,419.8800,None,15.000,524.8500000,19.0,NaN,166.2025000000,2.0666666667,668.3090000000
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,384.8900,209.9400,None,23.000,804.7700000,38.0,NaN,332.4050000000,1.9130434783,1179.1630000000
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1084.6900,384.8900,None,30.000,1049.7000000,23.0,NaN,201.1925000000,1.1333333333,1284.1330000000
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1539.5600,1084.6900,None,35.500,1242.1450000,2.0,NaN,17.4950000000,0.9295774648,1165.1670000000
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1189.6600,1539.5600,None,37.000,1294.6300000,-7.0,NaN,-61.2325000000,1.0000000000,1249.1430000000


Unfortunately, by default, the math transform creates the column name by simplifying the math operation carried out. This gives us the names:
* *ORDERQUANTITY_SUM___LAG_ORDERQUANTITY_SUM_3*
* *ORDERQUANTITY_SUM__NULLIFLAG_ORDERQUANTITY_SUM_12_0*
* *SALESAMOUNT_SUM___LAG_SALESAMOUNT_SUM_3__4*
* *SALESAMOUNT_SUM__NULLIFMEAN_SALESAMOUNT_SUM_4_0*
* *_4SALESAMOUNT_SUM__3LAG_SALESAMOUNT_SUM_1__2LAG_SALESAMOUNT_SUM_1__LAG_SALESAMOUNT_SUM_310*

These do not really represent the concepts well, so we will rename them using the `rename` transform to:
* *ORDERQUANTITY_SUM_DELTA_4*
* *ORDERQUANTITY_SUM_RATIO_12*
* *SALESAMOUNT_SUM_VELOCITY_4*
* *SALESAMOUNT_RATIO_MA_4*
* *SALESAMOUNT_SUM_WMA_4*

In [24]:
salesds = weeklysales.math(math_ops=['ORDERQUANTITY_SUM - LAG_ORDERQUANTITY_SUM_3',
                                     'ORDERQUANTITY_SUM / NULLIF(LAG_ORDERQUANTITY_SUM_12, 0)',
                                     '(SALESAMOUNT_SUM - LAG_SALESAMOUNT_SUM_3) / 4',
                                     'SALESAMOUNT_SUM / NULLIF(MEAN_SALESAMOUNT_SUM_4, 0)',
                                     '(4*SALESAMOUNT_SUM + 3*LAG_SALESAMOUNT_SUM_1 + 2*LAG_SALESAMOUNT_SUM_1 + LAG_SALESAMOUNT_SUM_3)/10']).rename(
                           renames={'ORDERQUANTITY_SUM___LAG_ORDERQUANTITY_SUM_3': 'ORDERQUANTITY_SUM_DELTA_4',
                                    'ORDERQUANTITY_SUM__NULLIFLAG_ORDERQUANTITY_SUM_12_0': 'ORDERQUANTITY_SUM_RATIO_12',
                                    'SALESAMOUNT_SUM___LAG_SALESAMOUNT_SUM_3__4': 'SALESAMOUNT_SUM_VELOCITY_4',
                                    'SALESAMOUNT_SUM__NULLIFMEAN_SALESAMOUNT_SUM_4_0': 'SALESAMOUNT_RATIO_MA_4',
                                    '_4SALESAMOUNT_SUM__3LAG_SALESAMOUNT_SUM_1__2LAG_SALESAMOUNT_SUM_1__LAG_SALESAMOUNT_SUM_310': 'SALESAMOUNT_SUM_WMA_4'})

salesds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,...,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4
0,NaN,NaN,None,1.0000000000,None,214,2012-12-23,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,4.000,139.9600000
1,NaN,NaN,None,1.4285714286,None,214,2012-12-30,0.0,0.0,0.0,...,34.9900000000,None,None,None,139.9600,None,None,None,7.000,244.9300000
2,NaN,NaN,None,1.3846153849,None,214,2013-01-06,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None,8.666,303.2466666
3,2.0,NaN,17.4950000000,0.7500000000,307.9120000000,214,2013-01-13,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None,8.000,279.9200000
4,1.0,NaN,8.7475000000,1.1282051282,293.9160000000,214,2013-01-20,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None,9.750,341.1525000
5,19.0,NaN,166.2025000000,2.0666666667,668.3090000000,214,2013-01-27,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None,15.000,524.8500000
6,38.0,NaN,332.4050000000,1.9130434783,1179.1630000000,214,2013-02-03,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None,23.000,804.7700000
7,23.0,NaN,201.1925000000,1.1333333333,1284.1330000000,214,2013-02-10,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None,30.000,1049.7000000
8,2.0,NaN,17.4950000000,0.9295774648,1165.1670000000,214,2013-02-17,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None,35.500,1242.1450000
9,-7.0,NaN,-61.2325000000,1.0000000000,1249.1430000000,214,2013-02-24,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None,37.000,1294.6300000


### Publish to Rasgo

At this point we've created all of the features from the internet sales data. We're not quite ready to model with it (we still need to merge in the product data and perform a last bit of feature engineering), but we'd like to make this work available to others and saved for future analysis. This means we will publish it to Rasgo.

In [25]:
finishedsales = salesds.save(table_name="WKSP_AW_WEEKLY_SALES_FORECASTING",
                           table_type="VIEW")
finishedsales

Dataset(fqtn=RASGOQL.PUBLIC.WKSP_AW_WEEKLY_SALES_FORECASTING, type=view)

## Product Data

Let's turn our attention to the product data. First, let's take a quick look again to remind ourselves what is here.

In [26]:
product.preview()

,PRODUCTKEY,PRODUCTALTERNATEKEY,PRODUCTSUBCATEGORYKEY,WEIGHTUNITMEASURECODE,SIZEUNITMEASURECODE,ENGLISHPRODUCTNAME,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,STANDARDCOST,FINISHEDGOODSFLAG,...,CHINESEDESCRIPTION,ARABICDESCRIPTION,HEBREWDESCRIPTION,THAIDESCRIPTION,GERMANDESCRIPTION,JAPANESEDESCRIPTION,TURKISHDESCRIPTION,STARTDATE,ENDDATE,STATUS
0,1,AR-5381,NaN,None,None,Adjustable Race,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
1,2,BA-8327,NaN,None,None,Bearing Ball,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
2,3,BE-2349,NaN,None,None,BB Ball Bearing,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
4,5,BL-2036,NaN,None,None,Blade,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
5,6,CA-5965,NaN,None,None,LL Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
6,7,CA-6738,NaN,None,None,ML Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
7,8,CA-7457,NaN,None,None,HL Crankarm,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
8,9,CB-2903,NaN,None,None,Chainring Bolts,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current
9,10,CN-6137,NaN,None,None,Chainring Nut,None,None,NaN,0,...,None,None,None,None,None,None,None,2003-07-01,NaT,Current


We see a lot of missing data, but looking closer *FINISHEDGOODSFLAG* is one, let's filter on this to see just finished goods. We can use the transform `filter` to filter the data.

In [27]:
finishedproducts = product.filter(filter_statements=["FINISHEDGOODSFLAG = 1"])
finishedproducts.preview()

,PRODUCTKEY,PRODUCTALTERNATEKEY,PRODUCTSUBCATEGORYKEY,WEIGHTUNITMEASURECODE,SIZEUNITMEASURECODE,ENGLISHPRODUCTNAME,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,STANDARDCOST,FINISHEDGOODSFLAG,...,CHINESEDESCRIPTION,ARABICDESCRIPTION,HEBREWDESCRIPTION,THAIDESCRIPTION,GERMANDESCRIPTION,JAPANESEDESCRIPTION,TURKISHDESCRIPTION,STARTDATE,ENDDATE,STATUS
0,210,FR-R92B-58,14,LB,CM,"HL Road Frame - Black, 58",None,None,NaN,1,...,重量最轻、质量最好的铝制车架采用最新的合金材质；经过焊接和热处理，坚固牢靠。我们创新的设计确...,لقد تم صناعة هيكل دراجتنا الألومنيوم الأخف وزن...,מסגרת האלומיניום הקלה והאיכותית ביותר שלנו עשו...,เฟรมอลูมิเนียมคุณภาพสูงสุดและน้ำหนักเบาที่สุด ...,Unser leichtester und hochwertigster Aluminium...,最新の合金で作られた最軽量/最高品質のアルミニウム フレームで、強度を得るために溶接および熱...,"""En yeni alaşımdan imal edilme, en hafif ve en...",2003-07-01,NaT,Current
1,211,FR-R92R-58,14,LB,CM,"HL Road Frame - Red, 58",None,None,NaN,1,...,重量最轻、质量最好的铝制车架采用最新的合金材质；经过焊接和热处理，坚固牢靠。我们创新的设计确...,لقد تم صناعة هيكل دراجتنا الألومنيوم الأخف وزن...,מסגרת האלומיניום הקלה והאיכותית ביותר שלנו עשו...,เฟรมอลูมิเนียมคุณภาพสูงสุดและน้ำหนักเบาที่สุด ...,Unser leichtester und hochwertigster Aluminium...,最新の合金で作られた最軽量/最高品質のアルミニウム フレームで、強度を得るために溶接および熱...,"""En yeni alaşımdan imal edilme, en hafif ve en...",2003-07-01,NaT,Current
2,212,HL-U509-R,31,None,None,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge",12.0278,1,...,通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن ب...,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצח...",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"""Einheitsgröße, atmungsaktiv, leicht, aufsetzb...",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2011-07-01,2007-12-28,None
3,213,HL-U509-R,31,None,None,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge",13.8782,1,...,通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن ب...,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצח...",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"""Einheitsgröße, atmungsaktiv, leicht, aufsetzb...",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2012-07-01,2008-12-27,None
4,214,HL-U509-R,31,None,None,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge",13.0863,1,...,通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن ب...,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצח...",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"""Einheitsgröße, atmungsaktiv, leicht, aufsetzb...",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2013-07-01,NaT,Current
5,215,HL-U509,31,None,None,"Sport-100 Helmet, Black","Casco deportivo: 100, negro","Casque sport 100, noir",12.0278,1,...,通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن ب...,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצח...",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"""Einheitsgröße, atmungsaktiv, leicht, aufsetzb...",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2011-07-01,2007-12-28,None
6,216,HL-U509,31,None,None,"Sport-100 Helmet, Black","Casco deportivo: 100, negro","Casque sport 100, noir",13.8782,1,...,通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن ب...,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצח...",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"""Einheitsgröße, atmungsaktiv, leicht, aufsetzb...",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2012-07-01,2008-12-27,None
7,217,HL-U509,31,None,None,"Sport-100 Helmet, Black","Casco deportivo: 100, negro","Casque sport 100, noir",13.0863,1,...,通用型透气良好且轻便，带有自合型帽沿。,ملائمة بشكل عام، وجيدة التهوية، وخفيفة الوزن ب...,"מידה אונברסלית, מאווררת היטב, קלת-משקל, עם מצח...",แว่นกันลมขนาดสากล ระบายอากาศได้ดี น้ำหนักเบา,"""Einheitsgröße, atmungsaktiv, leicht, aufsetzb...",軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2013-07-01,NaT,Current
8,218,SO-B909-M,23,Non

That looks better and we can use this data.

### Explore Product Subcategory

Promotion looks like it can just be added and only *DISCOUNTPCT* is needed. Product has a subcategory key and there is a relevant table ADVENTUREWORKS.PUBLIC.DIMPRODUCTSUBCATEGORY. Explore that

In [28]:
productsubcategory = rql.dataset('ADVENTUREWORKS.PUBLIC.DIMPRODUCTSUBCATEGORY')
productsubcategory.preview()

,PRODUCTSUBCATEGORYKEY,PRODUCTSUBCATEGORYALTERNATEKEY,ENGLISHPRODUCTSUBCATEGORYNAME,SPANISHPRODUCTSUBCATEGORYNAME,FRENCHPRODUCTSUBCATEGORYNAME,PRODUCTCATEGORYKEY
0,1,1,Mountain Bikes,Bicicleta de montaña,VTT,1
1,2,2,Road Bikes,Bicicleta de carretera,Vélo de route,1
2,3,3,Touring Bikes,Bicicleta de paseo,Vélo de randonnée,1
3,4,4,Handlebars,Barra,Barre d'appui,2
4,5,5,Bottom Brackets,Eje de pedalier,Axe de pédalier,2
5,6,6,Brakes,Frenos,Freins,2
6,7,7,Chains,Cadena,Chaîne,2
7,8,8,Cranksets,Bielas,Pédalier,2
8,9,9,Derailleurs,Desviador,Dérailleur,2
9,10,10,Forks,Horquilla,Fourche,2


In [29]:
productsubcategory.preview().columns.sort_values()

Index(['ENGLISHPRODUCTSUBCATEGORYNAME', 'FRENCHPRODUCTSUBCATEGORYNAME',
       'PRODUCTCATEGORYKEY', 'PRODUCTSUBCATEGORYALTERNATEKEY',
       'PRODUCTSUBCATEGORYKEY', 'SPANISHPRODUCTSUBCATEGORYNAME'],
      dtype='object')

### Join subcategory to product

We can use the `join` transformation to join the subcategory name to the product information.

In [30]:
finishedproducts2 = finishedproducts.join(join_table=productsubcategory.fqtn,
                                          join_type='LEFT',
                                          join_columns={'PRODUCTSUBCATEGORYKEY':'PRODUCTSUBCATEGORYKEY'})
finishedproducts2.preview()

,PRODUCTKEY,PRODUCTALTERNATEKEY,PRODUCTSUBCATEGORYKEY,WEIGHTUNITMEASURECODE,SIZEUNITMEASURECODE,ENGLISHPRODUCTNAME,SPANISHPRODUCTNAME,FRENCHPRODUCTNAME,STANDARDCOST,FINISHEDGOODSFLAG,...,JAPANESEDESCRIPTION,TURKISHDESCRIPTION,STARTDATE,ENDDATE,STATUS,PRODUCTSUBCATEGORYALTERNATEKEY,ENGLISHPRODUCTSUBCATEGORYNAME,SPANISHPRODUCTSUBCATEGORYNAME,FRENCHPRODUCTSUBCATEGORYNAME,PRODUCTCATEGORYKEY
0,210,FR-R92B-58,14,LB,CM,"HL Road Frame - Black, 58",None,None,NaN,1,...,最新の合金で作られた最軽量/最高品質のアルミニウム フレームで、強度を得るために溶接および熱...,"""En yeni alaşımdan imal edilme, en hafif ve en...",2003-07-01,NaT,Current,14,Road Frames,Cuadro de carretera,Cadre de vélo de route,2
1,211,FR-R92R-58,14,LB,CM,"HL Road Frame - Red, 58",None,None,NaN,1,...,最新の合金で作られた最軽量/最高品質のアルミニウム フレームで、強度を得るために溶接および熱...,"""En yeni alaşımdan imal edilme, en hafif ve en...",2003-07-01,NaT,Current,14,Road Frames,Cuadro de carretera,Cadre de vélo de route,2
2,212,HL-U509-R,31,None,None,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge",12.0278,1,...,軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2011-07-01,2007-12-28,None,31,Helmets,Casco,Casque,4
3,213,HL-U509-R,31,None,None,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge",13.8782,1,...,軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2012-07-01,2008-12-27,None,31,Helmets,Casco,Casque,4
4,214,HL-U509-R,31,None,None,"Sport-100 Helmet, Red","Casco deportivo: 100, rojo","Casque sport 100, rouge",13.0863,1,...,軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2013-07-01,NaT,Current,31,Helmets,Casco,Casque,4
5,215,HL-U509,31,None,None,"Sport-100 Helmet, Black","Casco deportivo: 100, negro","Casque sport 100, noir",12.0278,1,...,軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2011-07-01,2007-12-28,None,31,Helmets,Casco,Casque,4
6,216,HL-U509,31,None,None,"Sport-100 Helmet, Black","Casco deportivo: 100, negro","Casque sport 100, noir",13.8782,1,...,軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2012-07-01,2008-12-27,None,31,Helmets,Casco,Casque,4
7,217,HL-U509,31,None,None,"Sport-100 Helmet, Black","Casco deportivo: 100, negro","Casque sport 100, noir",13.0863,1,...,軽量で通気性に優れたユニバーサル フィットのスナップ オン バイザー。,"""Her bisiklete kolayca takılabilen, aerodinami...",2013-07-01,NaT,Current,31,Helmets,Casco,Casque,4
8,218,SO-B909-M,23,None,None,"Mountain Bike Socks, M","Calcetines para bicicleta de montaña, M","Chaussettes de VTT, taille M",3.3963,1,...,天然および合成繊維の組み合わせにより、湿気を防ぎ、適切な緩衝材を提供します。,Doğal ve sentetik fiber karışımı kuru kalır ve...,2011-07-01,2007-12-28,None,23,Socks,Calcetines,Chaussettes,3
9,219,SO-B909-L,23,None,None,"Mountain Bike Socks, L","Calcetines para bicicleta de montaña, G","Chaussettes de VTT, taille L",3.3963,1,...,天然および合成繊維の組み合わせにより、湿気を防ぎ、適切な緩衝材を提供します。,Doğal ve sentetik fiber karışımı kuru kalır ve...,2011-07-01,2007-12-28,None,23,Socks,Calcetines,Chaussettes,3


There are a lot of columns we don't really need, let's keep *PRODUCTKEY*, *CLASS*, *COLOR*, *DEALERPRICE*, *ENGLISHDESCRIPTION*, *ENGLISHPRODUCTNAME*, *ENGLISHPRODUCTSUBCATEGORYNAME*, and *STANDARDCOST*.

#### Drop unneeded columns

The transformation `drop_columns` can take either an **include_cols** or **exclude_cols** argument. As we know which columns we want to keep, **include_cols** will be easier.

We could run the transformation on the result of the last set, but these transformations can be chained together as follows.

In [31]:
finishedproducts2 = finishedproducts.join(join_table=productsubcategory.fqtn,
                                          join_type='LEFT',
                                          join_columns={'PRODUCTSUBCATEGORYKEY':'PRODUCTSUBCATEGORYKEY'}).drop_columns(
                                          include_cols=['PRODUCTKEY', 'CLASS', 'COLOR', 'DEALERPRICE', 'ENGLISHDESCRIPTION', 
                                                        'ENGLISHPRODUCTNAME', 'ENGLISHPRODUCTSUBCATEGORYNAME', 
                                                        'STANDARDCOST'])
finishedproducts2.preview()

,PRODUCTKEY,CLASS,COLOR,DEALERPRICE,ENGLISHDESCRIPTION,ENGLISHPRODUCTNAME,ENGLISHPRODUCTSUBCATEGORYNAME,STANDARDCOST
0,210,H,Black,NaN,Our lightest and best quality aluminum frame m...,"HL Road Frame - Black, 58",Road Frames,NaN
1,211,H,Red,NaN,Our lightest and best quality aluminum frame m...,"HL Road Frame - Red, 58",Road Frames,NaN
2,212,None,Red,20.1865,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,12.0278
3,213,None,Red,20.1865,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.8782
4,214,None,Red,20.9940,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
5,215,None,Black,20.1865,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Black",Helmets,12.0278
6,216,None,Black,20.1865,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Black",Helmets,13.8782
7,217,None,Black,20.9940,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Black",Helmets,13.0863
8,218,None,White,5.7000,Combination of natural and synthetic fibers st...,"Mountain Bike Socks, M",Socks,3.3963
9,219,None,White,5.7000,Combination of natural and synthetic fibers st...,"Mountain Bike Socks, L",Socks,3.3963


This looks like a useful table, we can publish it to allow us to reuse it in future analysis. 

In [32]:
finishedprod = finishedproducts2.save(table_name="WKSP_AW_PROD_DETAILS",
                                      table_type="VIEW")
finishedprod

Dataset(fqtn=RASGOQL.PUBLIC.WKSP_AW_PROD_DETAILS, type=view)

### Create Modeling Data

We can now join the product data to the sales data we have been working with

In [33]:
startingds = finishedsales.join(join_table=finishedprod.fqtn,
                                join_type='LEFT',
                                join_columns={'PRODUCTKEY': 'PRODUCTKEY'})
startingds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,...,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,CLASS,COLOR,DEALERPRICE,ENGLISHDESCRIPTION,ENGLISHPRODUCTNAME,ENGLISHPRODUCTSUBCATEGORYNAME,STANDARDCOST
0,NaN,NaN,None,1.0000000000,None,214,2012-12-23,0.0,0.0,0.0,...,None,4.000,139.9600000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
1,NaN,NaN,None,1.4285714286,None,214,2012-12-30,0.0,0.0,0.0,...,None,7.000,244.9300000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
2,NaN,NaN,None,1.3846153849,None,214,2013-01-06,0.0,0.0,0.0,...,None,8.666,303.2466666,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
3,2.0,NaN,17.4950000000,0.7500000000,307.9120000000,214,2013-01-13,0.0,0.0,0.0,...,None,8.000,279.9200000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
4,1.0,NaN,8.7475000000,1.1282051282,293.9160000000,214,2013-01-20,0.0,0.0,0.0,...,None,9.750,341.1525000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
5,19.0,NaN,166.2025000000,2.0666666667,668.3090000000,214,2013-01-27,0.0,0.0,0.0,...,None,15.000,524.8500000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
6,38.0,NaN,332.4050000000,1.9130434783,1179.1630000000,214,2013-02-03,0.0,0.0,0.0,...,None,23.000,804.7700000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
7,23.0,NaN,201.1925000000,1.1333333333,1284.1330000000,214,2013-02-10,0.0,0.0,0.0,...,None,30.000,1049.7000000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
8,2.0,NaN,17.4950000000,0.9295774648,1165.1670000000,214,2013-02-17,0.0,0.0,0.0,...,None,35.500,1242.1450000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863
9,-7.0,NaN,-61.2325000000,1.0000000000,1249.1430000000,214,2013-02-24,0.0,0.0,0.0,...,None,37.000,1294.6300000,None,Red,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863


To prepare this for modeling, we need to do three things. First, the target (next weeks sales needs to be created). Second, the categorical variables should be one-hot encoded. Finally, missing values should be imputed for the numeric columns.

#### Target Creation

Use the `lag` transform with a negative lag value to get next weeks sales as the target. While doing this, rename the value to make it clear that it is the target.

In [34]:
modelingds = startingds.lag(columns=['SALESAMOUNT_SUM'],
                            amounts=[-1],
                            order_by=['PRODUCTKEY', 'ORDERWEEK'],
                            partition=['PRODUCTKEY']).rename(
                            renames={'LAG_SALESAMOUNT_SUM__1': 'TARGET_SALESAMOUNT'})
modelingds.preview()

,TARGET_SALESAMOUNT,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,...,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,CLASS,COLOR,DEALERPRICE,ENGLISHDESCRIPTION,ENGLISHPRODUCTNAME,ENGLISHPRODUCTSUBCATEGORYNAME,STANDARDCOST
0,3374.9900,NaN,NaN,None,1.0000000000,None,349,2011-01-23,0.0,0.0,...,None,1.000,3374.9900000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
1,6749.9800,NaN,NaN,None,1.0000000000,None,349,2011-02-06,0.0,0.0,...,None,1.000,3374.9900000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
2,6749.9800,NaN,NaN,None,1.5000000000,None,349,2011-02-13,0.0,0.0,...,None,1.333,4499.9866666,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
3,3374.9900,1.0,NaN,843.7475000000,1.3333333333,6412.4810000000,349,2011-02-27,0.0,0.0,...,None,1.500,5062.4850000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
4,10124.9700,0.0,NaN,0E-10,0.6666666667,5062.4850000000,349,2011-03-20,0.0,0.0,...,None,1.500,5062.4850000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
5,3374.9900,1.0,NaN,843.7475000000,1.5000000000,6412.4810000000,349,2011-04-10,0.0,0.0,...,None,2.000,6749.9800000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
6,3374.9900,-1.0,NaN,-843.7475000000,0.5714285714,7087.4790000000,349,2011-05-01,0.0,0.0,...,None,1.750,5906.2325000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
7,3374.9900,0.0,NaN,0E-10,0.6666666667,3374.9900000000,349,2011-05-08,0.0,0.0,...,None,1.500,5062.4850000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
8,6749.9800,-2.0,NaN,-1687.4950000000,0.6666666667,4049.9880000000,349,2011-05-15,0.0,0.0,...,None,1.500,5062.4850000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944
9,10124.9700,1.0,NaN,843.7475000000,1.6000000000,4724.9860000000,349,2011-06-12,0.0,0.0,...,None,1.250,4218.7375000,H,Black,2024.994,Top-of-the-line competition mountain bike. Per...,"Mountain-100 Black, 42",Mountain Bikes,1898.0944


#### Categorical encoding

The columns that need to be encoded are: *CLASS*, *COLOR*, *ENGELISHPRODUCTNAME*, and *ENGLISHPRODUCTSUBCATEGORYNAME*. We will use the `one_hot_encode` transorm to encode *CLASS* and *COLOR*.

Since *ENGLISHPRODUCTNAME* and *ENGLISHPRODUCTSUBCATEGORYNAME* contain a large number of categorties and we intend to use tree-based modeling algorithms, we will encode *ENGLISHPRODUCTSUBCATEGORYNAME* with the `label_encode` transform. We will encode *ENGLISHPRODUCTNAME* with `target_encode` that will replace it by the mean target value of that category. Target encoding is a very powerful techinque to encode these high-cardinality categorical variables efficiently and help improve model performance.

In [35]:
modelingds = modelingds.one_hot_encode(column='CLASS').one_hot_encode(
                                       column='COLOR').target_encode(
                                       column='ENGLISHPRODUCTNAME',
                                       target='TARGET_SALESAMOUNT').label_encode(
                                       column='ENGLISHPRODUCTSUBCATEGORYNAME')

modelingds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ALL_VALUES_ARRAY,TARGET_SALESAMOUNT,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,...,DEALERPRICE,ENGLISHDESCRIPTION,ENGLISHPRODUCTNAME,ENGLISHPRODUCTSUBCATEGORYNAME,STANDARDCOST,CLASS_IS_NULL,COLOR_RED,COLOR_BLACK,ENGLISHPRODUCTNAME_TARGET_ENCODED,ENGLISHPRODUCTSUBCATEGORYNAME_ENCODED
0,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",349.9000,NaN,NaN,None,1.0000000000,None,214,2012-12-23,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
1,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",419.8800,NaN,NaN,None,1.4285714286,None,214,2012-12-30,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
2,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",209.9400,NaN,NaN,None,1.3846153849,None,214,2013-01-06,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
3,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",384.8900,2.0,NaN,17.4950000000,0.7500000000,307.9120000000,214,2013-01-13,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
4,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1084.6900,1.0,NaN,8.7475000000,1.1282051282,293.9160000000,214,2013-01-20,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
5,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1539.5600,19.0,NaN,166.2025000000,2.0666666667,668.3090000000,214,2013-01-27,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
6,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1189.6600,38.0,NaN,332.4050000000,1.9130434783,1179.1630000000,214,2013-02-03,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
7,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1154.6700,23.0,NaN,201.1925000000,1.1333333333,1284.1330000000,214,2013-02-10,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
8,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1294.6300,2.0,NaN,17.4950000000,0.9295774648,1165.1670000000,214,2013-02-17,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
9,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1014.7100,-7.0,NaN,-61.2325000000,1.0000000000,1249.1430000000,214,2013-02-24,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7


#### Imputation

As a final step before modeling, all numeric columns should have missing values replaced by a number. This can be done by the `impute` transformation. If a linear or logistic regression, SVM or neural network algorithm was going to be applied, we may want to impute the mean or median. This could be done by passing 'mean' or 'median' in through the imputations dictionary.

As the modeling algoritm applied here is tree-based, we can simply impute and extreme value. All of the features created are non-negative or close to zero, so we will impute a very large negative number, *-999,999*.

In [36]:
imputation_dict = {'DEALERPRICE': -999999,
                   'DISCOUNTAMOUNT_AVG': -999999,
                   'DISCOUNTAMOUNT_MAX': -999999,
                   'DISCOUNTAMOUNT_MIN': -999999,
                   'DISCOUNTAMOUNT_SUM': -999999,
                   'DISCOUNTPCT_AVG': -999999,
                   'DISCOUNTPCT_MAX': -999999,
                   'DISCOUNTPCT_MIN': -999999,
                   'DISCOUNTPCT_SUM': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_1': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_12': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_2': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_3': -999999,
                   'LAG_DISCOUNTPCT_AVG_1': -999999,
                   'LAG_DISCOUNTPCT_AVG_12': -999999,
                   'LAG_DISCOUNTPCT_AVG_2': -999999,
                   'LAG_DISCOUNTPCT_AVG_3': -999999,
                   'LAG_ORDERQUANTITY_SUM_1': -999999,
                   'LAG_ORDERQUANTITY_SUM_12': -999999,
                   'LAG_ORDERQUANTITY_SUM_2': -999999,
                   'LAG_ORDERQUANTITY_SUM_3': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_1': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_12': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_2': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_3': -999999,
                   'LAG_SALESAMOUNT_SUM_1': -999999,
                   'LAG_SALESAMOUNT_SUM_12': -999999,
                   'LAG_SALESAMOUNT_SUM_2': -999999,
                   'LAG_SALESAMOUNT_SUM_3': -999999,
                   'LAG_TAXAMT_SUM_1': -999999,
                   'LAG_TAXAMT_SUM_12': -999999,
                   'LAG_TAXAMT_SUM_2': -999999,
                   'LAG_TAXAMT_SUM_3': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_1': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_12': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_2': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_3': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_1': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_12': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_2': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_3': -999999,
                   'LAG_UNITPRICE_AVG_1': -999999,
                   'LAG_UNITPRICE_AVG_12': -999999,
                   'LAG_UNITPRICE_AVG_2': -999999,
                   'LAG_UNITPRICE_AVG_3': -999999,
                   'LAG_UNITPRICE_SUM_1': -999999,
                   'LAG_UNITPRICE_SUM_12': -999999,
                   'LAG_UNITPRICE_SUM_2': -999999,
                   'LAG_UNITPRICE_SUM_3': -999999,
                   'MEAN_ORDERQUANTITY_SUM_4': -999999,
                   'MEAN_SALESAMOUNT_SUM_4': -999999,
                   'ORDERQUANTITY_SUM': -999999,
                   'ORDERQUANTITY_SUM_DELTA_4': -999999,
                   'ORDERQUANTITY_SUM_RATIO_12': -999999,
                   'PRODUCTSTANDARDCOST_AVG': -999999,
                   'PRODUCTSTANDARDCOST_SUM': -999999,
                   'SALESAMOUNT_RATIO_MA_4': -999999,
                   'SALESAMOUNT_SUM': -999999,
                   'SALESAMOUNT_SUM_VELOCITY_4': -999999,
                   'SALESAMOUNT_SUM_WMA_4': -999999,
                   'STANDARDCOST': -999999,
                   'TAXAMT_SUM': -999999,
                   'TOTALPRODUCTCOST_AVG': -999999,
                   'TOTALPRODUCTCOST_SUM': -999999,
                   'UNITPRICEDISCOUNTPCT_AVG': -999999,
                   'UNITPRICEDISCOUNTPCT_MAX': -999999,
                   'UNITPRICEDISCOUNTPCT_MIN': -999999,
                   'UNITPRICEDISCOUNTPCT_SUM': -999999,
                   'UNITPRICE_AVG': -999999,
                   'UNITPRICE_SUM': -999999}

In [37]:
modelingds = modelingds.impute(imputations=imputation_dict)

modelingds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ALL_VALUES_ARRAY,TARGET_SALESAMOUNT,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,...,DEALERPRICE,ENGLISHDESCRIPTION,ENGLISHPRODUCTNAME,ENGLISHPRODUCTSUBCATEGORYNAME,STANDARDCOST,CLASS_IS_NULL,COLOR_RED,COLOR_BLACK,ENGLISHPRODUCTNAME_TARGET_ENCODED,ENGLISHPRODUCTSUBCATEGORYNAME_ENCODED
0,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",349.9000,-999999,-999999.0,-999999.0000000000,1.0000000000,-999999.0000000000,214,2012-12-23,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
1,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",419.8800,-999999,-999999.0,-999999.0000000000,1.4285714286,-999999.0000000000,214,2012-12-30,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
2,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",209.9400,-999999,-999999.0,-999999.0000000000,1.3846153849,-999999.0000000000,214,2013-01-06,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
3,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",384.8900,2,-999999.0,17.4950000000,0.7500000000,307.9120000000,214,2013-01-13,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
4,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1084.6900,1,-999999.0,8.7475000000,1.1282051282,293.9160000000,214,2013-01-20,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
5,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1539.5600,19,-999999.0,166.2025000000,2.0666666667,668.3090000000,214,2013-01-27,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
6,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1189.6600,38,-999999.0,332.4050000000,1.9130434783,1179.1630000000,214,2013-02-03,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
7,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1154.6700,23,-999999.0,201.1925000000,1.1333333333,1284.1330000000,214,2013-02-10,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
8,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1294.6300,2,-999999.0,17.4950000000,0.9295774648,1165.1670000000,214,2013-02-17,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7
9,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1014.7100,-7,-999999.0,-61.2325000000,1.0000000000,1249.1430000000,214,2013-02-24,0.0,...,20.994,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7


#### Train-test split

As this is a time-series problem, a random train-test split won't work as there will be leakage from observations near the end of the time frame in the training set to observations earlier than this in the test set. The way to avoid this problem is to perform the split based on the date. The transformation `train_test_split` can do this by passing the date columns through the parameter **order_by**.

In [38]:
modelingds = modelingds.train_test_split(order_by=['ORDERWEEK'],
                                         train_percent=0.8)
    
modelingds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ALL_VALUES_ARRAY,TARGET_SALESAMOUNT,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,...,ENGLISHDESCRIPTION,ENGLISHPRODUCTNAME,ENGLISHPRODUCTSUBCATEGORYNAME,STANDARDCOST,CLASS_IS_NULL,COLOR_RED,COLOR_BLACK,ENGLISHPRODUCTNAME_TARGET_ENCODED,ENGLISHPRODUCTSUBCATEGORYNAME_ENCODED,TT_SPLIT
0,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",349.9000,-999999,-999999.0,-999999.0000000000,1.0000000000,-999999.0000000000,214,2012-12-23,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
1,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",419.8800,-999999,-999999.0,-999999.0000000000,1.4285714286,-999999.0000000000,214,2012-12-30,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
2,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",209.9400,-999999,-999999.0,-999999.0000000000,1.3846153849,-999999.0000000000,214,2013-01-06,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
3,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",384.8900,2,-999999.0,17.4950000000,0.7500000000,307.9120000000,214,2013-01-13,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
4,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1084.6900,1,-999999.0,8.7475000000,1.1282051282,293.9160000000,214,2013-01-20,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
5,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1539.5600,19,-999999.0,166.2025000000,2.0666666667,668.3090000000,214,2013-01-27,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
6,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1189.6600,38,-999999.0,332.4050000000,1.9130434783,1179.1630000000,214,2013-02-03,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
7,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1154.6700,23,-999999.0,201.1925000000,1.1333333333,1284.1330000000,214,2013-02-10,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
8,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1294.6300,2,-999999.0,17.4950000000,0.9295774648,1165.1670000000,214,2013-02-17,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN
9,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1014.7100,-7,-999999.0,-61.2325000000,1.0000000000,1249.1430000000,214,2013-02-24,0.0,...,"Universal fit, well-vented, lightweight , snap...","Sport-100 Helmet, Red",Helmets,13.0863,1,1,0,1366.452,7,TRAIN


#### Delete unneeded columns

We now have a number of columns not needed for modeling (such as the raw categorical columns), we can delete the following from the dataset.
* *CLASS*
* *COLOR*
* *ENGLISHDESCRIPTION*
* *ENGLISHPRODUCTNAME*
* *ENGLISHPRODUCTSUBCATEGORYNAME*


In [39]:
modelingds = modelingds.drop_columns(exclude_cols=['CLASS', 'COLOR', 'ENGLISHDESCRIPTION', 
                                                   'ENGLISHPRODUCTNAME', 'ENGLISHPRODUCTSUBCATEGORYNAME'])
    
modelingds.order(order_by={'PRODUCTKEY':'ASC','ORDERWEEK':'ASC'}).preview()

,ALL_VALUES_ARRAY,TARGET_SALESAMOUNT,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,...,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,DEALERPRICE,STANDARDCOST,CLASS_IS_NULL,COLOR_RED,COLOR_BLACK,ENGLISHPRODUCTNAME_TARGET_ENCODED,ENGLISHPRODUCTSUBCATEGORYNAME_ENCODED,TT_SPLIT
0,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",349.9000,-999999,-999999.0,-999999.0000000000,1.0000000000,-999999.0000000000,214,2012-12-23,0.0,...,4.000,139.9600000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
1,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",419.8800,-999999,-999999.0,-999999.0000000000,1.4285714286,-999999.0000000000,214,2012-12-30,0.0,...,7.000,244.9300000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
2,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",209.9400,-999999,-999999.0,-999999.0000000000,1.3846153849,-999999.0000000000,214,2013-01-06,0.0,...,8.666,303.2466666,20.994,13.0863,1,1,0,1366.452,7,TRAIN
3,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",384.8900,2,-999999.0,17.4950000000,0.7500000000,307.9120000000,214,2013-01-13,0.0,...,8.000,279.9200000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
4,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1084.6900,1,-999999.0,8.7475000000,1.1282051282,293.9160000000,214,2013-01-20,0.0,...,9.750,341.1525000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
5,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1539.5600,19,-999999.0,166.2025000000,2.0666666667,668.3090000000,214,2013-01-27,0.0,...,15.000,524.8500000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
6,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1189.6600,38,-999999.0,332.4050000000,1.9130434783,1179.1630000000,214,2013-02-03,0.0,...,23.000,804.7700000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
7,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1154.6700,23,-999999.0,201.1925000000,1.1333333333,1284.1330000000,214,2013-02-10,0.0,...,30.000,1049.7000000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
8,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1294.6300,2,-999999.0,17.4950000000,0.9295774648,1165.1670000000,214,2013-02-17,0.0,...,35.500,1242.1450000,20.994,13.0863,1,1,0,1366.452,7,TRAIN
9,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",1014.7100,-7,-999999.0,-61.2325000000,1.0000000000,1249.1430000000,214,2013-02-24,0.0,...,37.000,1294.6300000,20.994,13.0863,1,1,0,1366.452,7,TRAIN


#### Save Modeling Dataset

We can now save this modeling dataset so we can return to it in the future.

In [40]:
modeling = modelingds.save(table_name="WKSP_AW_SALES_FORECASTION",
                           table_type="VIEW")
modeling

Dataset(fqtn=RASGOQL.PUBLIC.WKSP_AW_SALES_FORECASTION, type=view)

Grab the table name for production use.

In [41]:
modeling_fqtn = modeling.fqtn

### Modeling

We are now ready to build the model. First, get the modeling data from Rasgo using `to_df`.

In [42]:
df = modeling.to_df().reset_index(drop=True)

Check for numeric datatypes and convert the numeric ones to floats.

In [43]:
for c in df.select_dtypes(exclude=[np.number]).columns:
    if c not in ['ORDERWEEK', 'TT_SPLIT', 'ALL_VALUES_ARRAY']:
        df[c] = pd.to_numeric(df[c])

Eliminate the last week of data as there is no target.

In [44]:
df = df[~df.TARGET_SALESAMOUNT.isna()]

#### Train the model

First, split the data using the TT_SPLIT column.

In [45]:
df_train = df[df['TT_SPLIT'] == 'TRAIN'].drop(columns=['TT_SPLIT', 'ORDERWEEK', 'ALL_VALUES_ARRAY'])
df_test = df[df['TT_SPLIT'] == 'TEST'].drop(columns=['TT_SPLIT', 'ORDERWEEK', 'ALL_VALUES_ARRAY'])

In [46]:
y_train = df_train['TARGET_SALESAMOUNT']
X_train = df_train.drop(columns=['TARGET_SALESAMOUNT'])
y_test = df_test['TARGET_SALESAMOUNT']
X_test = df_test.drop(columns=['TARGET_SALESAMOUNT'])

#### Fit the model

For illustration purposes, we are just fitting the model with a single set of parameters. In general, you should optimize the hyperparameters before building the final model. That process is beyond the scope of this document.

In [47]:
model = xgb.XGBRegressor(n_estimators=100,
                         max_depth=5,
                         eta=0.01,
                         random_state=1066,
                         subsample=0.7,
                         colsample_bytree=0.7)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, eta=0.01, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.00999999978, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=1066,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### Check the performance

In [48]:
model.predict(X_test)

array([12026.676   ,   486.42685 ,   813.27313 , ...,   319.27356 ,
         113.257774,   391.7992  ], dtype=float32)

In [49]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse

4785.099860242459

### Predict on new data

Since our feature engineering was saved in Rasgo, as new data enters the system, it will automatically be prepared for modeling. We can just pull the data in question and make a prediction on it.

In this case, if we are making these predictions each week, we can just pull the most recent week. In this particular data, that is '*2014-01-19*'.

#### Pull the data

Use to_df to grab the data from this date. We have several columns not needed in the model, so we will drop those as well.

In [50]:
predictdf = rql.dataset(modeling_fqtn).filter(filter_statements=["ORDERWEEK = '2014-01-19'"]).to_df()
for c in predictdf.select_dtypes(exclude=[np.number]).columns:
    if c not in ['ORDERWEEK', 'TT_SPLIT', 'ALL_VALUES_ARRAY']:
        predictdf[c] = pd.to_numeric(predictdf[c])
predictdf.head()

,ALL_VALUES_ARRAY,TARGET_SALESAMOUNT,ORDERQUANTITY_SUM_DELTA_4,ORDERQUANTITY_SUM_RATIO_12,SALESAMOUNT_SUM_VELOCITY_4,SALESAMOUNT_RATIO_MA_4,SALESAMOUNT_SUM_WMA_4,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,...,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4,DEALERPRICE,STANDARDCOST,CLASS_IS_NULL,COLOR_RED,COLOR_BLACK,ENGLISHPRODUCTNAME_TARGET_ENCODED,ENGLISHPRODUCTSUBCATEGORYNAME_ENCODED,TT_SPLIT
0,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",419.88,-4,0.400000,-34.990,0.888889,731.291,217,2014-01-19,0.0,...,20.25,708.5475,20.994,13.0863,1,0,1,1277.442,7,TEST
1,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",53.99,-2,0.250000,-26.995,0.571429,64.788,490,2014-01-19,0.0,...,1.75,94.4825,32.394,41.5723,1,0,0,358.648,9,TEST
2,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",84.83,-14,0.459459,-17.465,0.800000,196.606,528,2014-01-19,0.0,...,42.50,212.0750,2.994,1.8663,1,0,0,270.773,14,TEST
3,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",24.49,0,0.500000,0.000,1.120000,134.695,465,2014-01-19,0.0,...,6.25,153.0625,14.694,9.1593,1,0,1,225.399,6,TEST
4,"[\n ""Bike Racks"",\n ""Bike Stands"",\n ""Bottl...",149.97,4,0.777778,49.990,1.333333,304.939,231,2014-01-19,0.0,...,5.25,262.4475,29.994,38.4923,1,0,0,400.829,9,TEST


Now use the model to get the sales forecast. We will create a dataframe to hold the predictions then drop the columns not needed by the model before making the prediction.

In [51]:
salesforecastdf = predictdf[['PRODUCTKEY', 'ORDERWEEK']].copy()
salesforecastdf['forecast'] = model.predict(predictdf.drop(columns=['TT_SPLIT', 'ORDERWEEK', 'ALL_VALUES_ARRAY', 'TARGET_SALESAMOUNT']))
salesforecastdf

,PRODUCTKEY,ORDERWEEK,forecast
0,217,2014-01-19,594.836670
1,490,2014-01-19,132.517212
2,528,2014-01-19,137.050568
3,465,2014-01-19,115.798225
4,231,2014-01-19,209.315765
5,477,2014-01-19,137.939590
6,482,2014-01-19,109.305435
7,475,2014-01-19,261.940033
8,491,2014-01-19,179.262054
9,471,2014-01-19,113.257774
